In [14]:
#!/usr/bin/env python

from dask.distributed import Client, progress
# HPC
# client = Client(scheduler_file='/glade/scratch/jhamman/scheduler.json')
# client

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=4)
cluster


cluster

/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [15]:

client = Client(cluster)

client

Client Scheduler: tcp://10.32.15.48:41951 Dashboard: /user/0000-0001-7783-5629/proxy/36055/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [16]:
import xarray as xr
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client
import numpy as np                                                                                        
import zarr


In [17]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import xscale 
import Wavenum_freq_spec_func as wfs
import time
import gcsfs

import matplotlib.pyplot as plt


In [18]:
%time

fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')
gcs = gcsfs.GCSFileSystem(gcs=fs,check=False,create=False)

mapzarr = gcs.get_mapper('pangeo-data/eNATL60-BLBT02X-ssh')
ds = xr.open_zarr(mapzarr)



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs


In [19]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 11688, x: 8354, y: 4729)
Coordinates:
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2010-01-19T00:30:00 ... 2009-09-17T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    nav_lat               (y, x) float32 dask.array<chunksize=(240, 480), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(240, 480), meta=np.ndarray>
    sossheig              (time_counter, y, x) float32 dask.array<chunksize=(240, 240, 480), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(240, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(240, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    TimeStamp:    24/03/2019 14:43:29 +0100
    descr

In [20]:
g = zarr.open_group(mapzarr)
g.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,fsspec.mapping.FSMap
No. members,8
No. arrays,7
No. groups,1
Arrays,"nav_lat, nav_lon, sossheig, time_centered, time_centered_bounds, time_counter, time_counter_bounds"
Groups,NATL60


In [21]:
g.sossheig.info

Name,/sossheig
Type,zarr.core.Array
Data type,float32
Shape,"(11688, 4729, 8354)"
Chunk shape,"(240, 240, 480)"
Order,C
Read-only,False
Compressor,"Blosc(cname='snappy', clevel=6, shuffle=AUTOSHUFFLE, blocksize=0)"
Store type,fsspec.mapping.FSMap
No. bytes,1846987597632 (1.7T)
Chunks initialized,15919/17640


In [22]:
ds.sossheig.encoding

{'chunks': (240, 240, 480),
 'compressor': Blosc(cname='snappy', clevel=6, shuffle=AUTOSHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float32'),
 'coordinates': 'time_centered'}

In [23]:
date='2009-10-01T00:30:00'
lat=ds.nav_lat.values
lon=ds.nav_lon.values


data=ds.sel(time_counter=date)['sossheig']


In [ ]:
fig=plt.figure(figsize=(20,15))
data.plot()

In [ ]:
client.close()

In [ ]:
cluster.close()